# Deep Learning for NLP

In the earlier section, we used classical machine learning techniques to build our text classifiers. In this chapter, we will replace those with deep learning techniques: Recurrent Neural Networks. 

In particular, we will use a relatively simple BiDirectional LSTM model. If this is new to you, keep reading - if not, please feel free to skip ahead! 

I begin by touching upon the overhyped terms as 'deep' in Deep Learning and 'neural' in Deep Neural Networks. I do a quick detour to why I use PyTorch and compare it to Tensorflow and Keras - the other popular Deep Learning frameworks.   

I will build the simplest possible architecture for demonstration here. I assume a general familiarity with RNNs and don’t introduce the same again.  

In this section, we answer the following questions: 
- What is Deep Learning? How does it differ from what we have seen? 
- What are the key ideas in any deep learning model? 
- Why PyTorch?  
- How to tokenize text and setting up dataloaders with `torchtext`?
- What recurrent networks are, and how to use them for text classification? 

# What is Deep Learning? 

Deep learning is a subset of machine learning: a new take on learning from data that puts an emphasis on learning successive layers of increasingly meaningful representations. But what does 'deep' in Deep Learning mean? 

> The deep in deep learning isn’t a reference to any kind of deeper understanding achieved by the approach; rather, it stands for this idea of successive layers of representations. - F. Chollet, Lead Developer of Keras

The _depth_ of the model is indicative of how many layers of such representations did we use. F Chollet suggested _layered representations learning_, _hierarchical representations learning_ as better names for these. Another name could have been _differentiable programming_. This term coined by Yann LeCun, reasons from that the common thing between our 'deep learning methods' are not more layers. Instead, that all these models learn via some form of differential calculus—most often stochastic gradient descent.

## Differences with 'Modern' Machine Learning Methods
The modern machine learning methods which we saw shot to mainstream mostly in the 1990s or after that. The binding factor among them was that they all use one layer of representations. For instance, the Decision Trees just create one set of rules and apply them. Even if you add ensemble approaches, the 'ensembling' is often shallow and only combines several ML models directly.

Here is a better worded interpretation of these differences: 

> Modern deep learning often involves tens or even hundreds of successive layers of representations—and they’re all learned automatically from exposure to training data. Meanwhile, other approaches to machine learning tend to focus on learning only one or two layers of representations of the data; hence, they’re sometimes called shallow learning. - F Chollet

# Understanding Deep Learning

In a loosely worded manner, machine learning is about mapping inputs (such as image, or "movie review") to targets (such as the label cat or “positive”). The model does this by looking at (or training from) several pairs of input and targets. 

Deep Neural Networks do this input-to-target mapping using a long sequence of simple data transformations (layers). This sequence length is referred to as depth of the network. The entire sequence from input-to-target is referred to as _model_ which learns about the data. These data transformations are learned by repeated obvservation of examples.  let’s look at how this learning happens, concretely.

As a necessary caveat here, the _neural_ in Neural Networks has nothing to do with human brain except serving as a bad metaphor. There are several articles written by lazy journalists who did not bother asking any actual Deep Learning engineer or researcher about this term. You can safely ignore _all of them_ 

## Puzzle Pieces

We are looking at a particular sub-class of challenges where we want to learn an input-to-target mapping. This subclass is generally referred to as supervised machine learning. The word _supervised_ denoting that we have target(s) for each input. Unsupervised machine learning includes challenges like trying to cluster text, where we do not have a target. 

In order to do any supervised machine learning, we need the following in-place: 

 1. Input Data : Anything ranging from past stock performance to your vacation pictures 
 1. Target - Examples of the expected output:
 1. A way to measure whether the algorithm is doing a good job — This is necessary in order to determine the distance between the algorithm’s current output and its expected output. 
 
The above components are universal to any supervised approach, machine learning or deep learning. Deep Learning in particular has it's own starcast of puzzle pieces: 

1. Model Itself  
1. Loss Function
1. Optimizer

Since these actors are new to the scene, let's take a minute in understanding what they do: 

### Model

Each model is comprised of several layers. Each layer is a data transformation. The transformation is captured using a bunch of numbers - called layer weights or parameters.

The specification of what a layer does to its input data is stored in the layer’s weights, which in essence are a bunch of numbers. In technical terms, we’d say that the transformation implemented by a layer is parameterized by its weights (see figure 1.7). (Weights are also sometimes called the parameters of a layer.) In this context, learning means finding a set of values for the weights of all layers in a network, such that the network will correctly map example inputs to their associated targets. But here’s the thing: a deep neural network can contain tens of millions of parameters. Finding the correct value for all of them may seem like a daunting task, especially given that modifying the value of one parameter will affect the behavior of all the others!

### Loss Function
This is somt the _loss function_ in context of Deep Learning.
 
The measurement is _automatically_ used as a feedback signal to adjust the way the algorithm works. This adjustment step is what we call learning.

### Optimizer

# Kaggle: Text Categorization Challenge

In this particular section, we are going to visit the familiar task of text classification. We are going to use a different datset though. We are going to be solving the [Jigsaw Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge).

# Getting the Data

Note that you will need to do accept the terms and conditions of the competition and data usage in order to get this dataset.

**Direct Download**: You can get the train and test data from the [data tab on challenge website](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data).  

**Kaggle API**: You can use the official Kaggle API [(github link)](https://github.com/Kaggle/kaggle-api) to download the data

In case of direct download and Kaggle API both, you have to split your train data into smaller train and validation splits for this notebook. 

You can create train and valid splits of train data using `sklearn.model_selection.train_test_split` utility. Alternatively, you can download directly from the accompanying ...

**Github Repo**: I am uploading the exact splits that I am using to a repository associated with this codebase. 

## Exploring the Data

In [1]:
# !conda install -y pandas
# !conda install -y numpy

In [2]:
import pandas as pd
import numpy as np

In [3]:
train_df = pd.read_csv("data/train.csv")

In [4]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
val_df = pd.read_csv("data/valid.csv")

In [6]:
val_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,000eefc67a2c930f,Radial symmetry \r\n\r\nSeveral now extinct li...,0,0,0,0,0,0
1,000f35deef84dc4a,There's no need to apologize. A Wikipedia arti...,0,0,0,0,0,0
2,000ffab30195c5e1,"Yes, because the mother of the child in the ca...",0,0,0,0,0,0
3,0010307a3a50a353,"""\r\nOk. But it will take a bit of work but I ...",0,0,0,0,0,0
4,0010833a96e1f886,"""== A barnstar for you! ==\r\n\r\n The Real L...",0,0,0,0,0,0


In [7]:
test_df = pd.read_csv("data/test.csv")

In [8]:
test_df.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \r\n\r\n The title is fine as i...
2,00013b17ad220c46,""" \r\n\r\n == Sources == \r\n\r\n * Zawe Ashto..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


#TODO: Explain the notion of target and that we have to predict 6 target labels here


Factor|Tensorflow |PyTorch|
--|--|--|
Works| No| Yes|


Tensorflow has a much bigger community behind it than PyTorch. This means that it becomes easier to find resources to learn Tensorflow and also, to find solutions to your problems. Also, many tutorials and MOOCs cover Tensorflow instead of using PyTorch. This is because PyTorch is a relatively new framework as compared to Tensorflow. So, in terms of resources, you will find much more content about Tensorflow than PyTorch.Tensorflow has a much bigger community behind it than PyTorch. This means that it becomes easier to find resources to learn Tensorflow and also, to find solutions to your problems. Also, many tutorials and MOOCs cover Tensorflow instead of using PyTorch. This is because PyTorch is a relatively new framework as compared to Tensorflow. So, in terms of resources, you will find much more content about Tensorflow than PyTorch.

## PyTorch and torchtext

In [9]:
# !pip install --upgrade git+https://github.com/pytorch/text

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext

In [11]:
use_gpu = True
if use_gpu:
    assert torch.cuda.is_available()

In [12]:
torch.cuda.device_count()

1

#TODO Write how writing good data loaders is the most tedious part in most deep learning applications
#TODO Start a new section here

The Field class determines how the data is preprocessed and converted into a numeric format

Based on [Practical Torchtext](https://github.com/keitakurita/practical-torchtext/) by Keita Kurita

In [13]:
from torchtext.data import Field

In [14]:
tokenize = lambda x: x.split()
TEXT = Field(sequential=True, tokenize=tokenize, lower=True)

In [15]:
LABEL = Field(sequential=False, use_vocab=False)

In [16]:
from torchtext.data import TabularDataset

In [17]:
%%time
tv_datafields = [("id", None), # we won't be needing the id, so we pass in None as the field
                 ("comment_text", TEXT), ("toxic", LABEL),
                 ("severe_toxic", LABEL), ("threat", LABEL),
                 ("obscene", LABEL), ("insult", LABEL),
                 ("identity_hate", LABEL)]

trn, vld = TabularDataset.splits(
        path="data", # the root directory where the data lies
        train='train.csv', validation="valid.csv",
        format='csv',
        skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
        fields=tv_datafields)

Wall time: 3.99 ms


In [18]:
%%time
tst_datafields = [("id", None), # we won't be needing the id, so we pass in None as the field
                 ("comment_text", TEXT)
]

tst = TabularDataset(
        path="data/test.csv", # the file path
        format='csv',
        skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
        fields=tst_datafields)

Wall time: 997 µs


In [19]:
TEXT.build_vocab(trn)

In [20]:
TEXT.vocab.freqs.most_common(10)

[('the', 78),
 ('to', 41),
 ('you', 33),
 ('of', 30),
 ('and', 26),
 ('a', 26),
 ('is', 24),
 ('that', 22),
 ('i', 20),
 ('if', 19)]

In [21]:
trn[0]

In [22]:
trn[0].__dict__.keys()

dict_keys(['comment_text', 'toxic', 'severe_toxic', 'threat', 'obscene', 'insult', 'identity_hate'])

In [23]:
trn[4].comment_text

['you,',
 'sir,',
 'are',
 'my',
 'hero.',
 'any',
 'chance',
 'you',
 'remember',
 'what',
 'page',
 "that's",
 'on?']

## Iterators!

In [24]:
from torchtext.data import Iterator, BucketIterator

In [25]:
train_iter, val_iter = BucketIterator.splits(
        (trn, vld), # we pass in the datasets we want the iterator to draw data from
        batch_sizes=(32, 32),
        device=0, # if you want to use the CPU, specify -1 or GPU ID here
        sort_key=lambda x: len(x.comment_text), # the BucketIterator needs to be told what function it should use to group the data.
        sort_within_batch=False,
        repeat=False # we pass repeat=False because we want to wrap this Iterator layer.
)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [26]:
train_iter

In [27]:
batch = next(train_iter.__iter__()); batch


[torchtext.data.batch.Batch of size 25]
	[.comment_text]:[torch.LongTensor of size 494x25]
	[.toxic]:[torch.LongTensor of size 25]
	[.severe_toxic]:[torch.LongTensor of size 25]
	[.threat]:[torch.LongTensor of size 25]
	[.obscene]:[torch.LongTensor of size 25]
	[.insult]:[torch.LongTensor of size 25]
	[.identity_hate]:[torch.LongTensor of size 25]

In [28]:
batch.__dict__.keys()

dict_keys(['batch_size', 'dataset', 'fields', 'comment_text', 'toxic', 'severe_toxic', 'threat', 'obscene', 'insult', 'identity_hate'])

In [29]:
test_iter = Iterator(tst, batch_size=64, device=0, sort=False, sort_within_batch=False, repeat=False)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [30]:
next(test_iter.__iter__())


[torchtext.data.batch.Batch of size 33]
	[.comment_text]:[torch.LongTensor of size 158x33]

In [31]:
class BatchWrapper:
    def __init__(self, dl, x_var, y_vars):
        self.dl, self.x_var, self.y_vars = dl, x_var, y_vars # we pass in the list of attributes for x and y
    
    def __iter__(self):
        for batch in self.dl:
            x = getattr(batch, self.x_var) # we assume only one input in this wrapper
            
            if self.y_vars is not None: # we will concatenate y into a single tensor
                y = torch.cat([getattr(batch, feat).unsqueeze(1) for feat in self.y_vars], dim=1).float()
            else:
                y = torch.zeros((1))
            if use_gpu:
                yield (x.cuda(), y.cuda())
            else:
                yield (x, y)
    
    def __len__(self):
        return len(self.dl)

In [32]:
train_dl = BatchWrapper(train_iter, "comment_text", ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])
valid_dl = BatchWrapper(val_iter, "comment_text", ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])
test_dl = BatchWrapper(test_iter, "comment_text", None)

In [33]:
next(train_dl.__iter__())

(tensor([[ 778,   15,  315,  ...,   15,  660,  280],
         [ 650,  601,   12,  ...,   46,   11,   18],
         [  22,  242,    6,  ...,   10,    2,   14],
         ...,
         [   1,    1,    1,  ...,    1,    1,    1],
         [   1,    1,    1,  ...,    1,    1,    1],
         [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0'),
 tensor([[ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 1.,  1.,  0.,  1.,  1.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,

## Training a Text Classifier

In [34]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [35]:
class SimpleBiLSTMBaseline(nn.Module):
    def __init__(self, hidden_dim, emb_dim=300,
                 spatial_dropout=0.05, recurrent_dropout=0.1, num_linear=2):
        super().__init__() # don't forget to call this!
        self.embedding = nn.Embedding(len(TEXT.vocab), emb_dim)
        self.encoder = nn.LSTM(emb_dim, hidden_dim, num_layers=1, dropout=recurrent_dropout)
        self.linear_layers = []
        for _ in range(num_linear - 1):
            self.linear_layers.append(nn.Linear(hidden_dim, hidden_dim))
        self.linear_layers = nn.ModuleList(self.linear_layers)
        self.predictor = nn.Linear(hidden_dim, 6)
    
    def forward(self, seq):
        hdn, _ = self.encoder(self.embedding(seq))
        feature = hdn[-1, :, :]
        for layer in self.linear_layers:
            feature = layer(feature)
        preds = self.predictor(feature)
        return preds

# Bi-LSTM Classifiers
- What is a Bi-LSTM? 
- What is a RNN? 
- Implement LSTM-only classification example
- Implement Bi-LSTM classification example

In [36]:
em_sz = 100
nh = 500
nl = 3
model = SimpleBiLSTMBaseline(nh, emb_dim=em_sz)
print(model)

D:\Miniconda3\envs\nlp\lib\site-packages\torch\nn\modules\rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


SimpleBiLSTMBaseline(
  (embedding): Embedding(784, 100)
  (encoder): LSTM(100, 500, dropout=0.1)
  (linear_layers): ModuleList(
    (0): Linear(in_features=500, out_features=500, bias=True)
  )
  (predictor): Linear(in_features=500, out_features=6, bias=True)
)


In [37]:
if use_gpu:
    model = model.cuda()
from tqdm import tqdm

In [38]:
opt = optim.Adam(model.parameters(), lr=1e-2)
loss_func = nn.BCEWithLogitsLoss().cuda()

In [39]:
epochs = 4

In [40]:
for epoch in range(1, epochs + 1):
    running_loss = 0.0
    running_corrects = 0
    model.train() # turn on training mode
    for x, y in tqdm(train_dl): # thanks to our wrapper, we can intuitively iterate over our data!
        opt.zero_grad()

        preds = model(x)
        loss = loss_func(preds, y)
        loss.backward()
        opt.step()
        
        running_loss += loss.item() * x.size(0)
        
    epoch_loss = running_loss / len(trn)
    
    # calculate the validation loss for this epoch
    val_loss = 0.0
    model.eval() # turn on evaluation mode
    for x, y in valid_dl:
        preds = model(x)
        loss = loss_func(preds, y)
        val_loss += loss.item() * x.size(0)

    val_loss /= len(vld)
    print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'.format(epoch, epoch_loss, val_loss))

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.99s/it]


Epoch: 1, Training Loss: 13.8331, Validation Loss: 4.7092


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.03it/s]


Epoch: 2, Training Loss: 7.5630, Validation Loss: 26.1762


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.55it/s]


Epoch: 3, Training Loss: 61.3190, Validation Loss: 2.6969


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11.02it/s]


Epoch: 4, Training Loss: 6.6757, Validation Loss: 2.1222


In [41]:
test_preds = []
for x, y in tqdm(test_dl):
    preds = model(x)
    # if you're data is on the GPU, you need to move the data back to the cpu
    # preds = preds.data.cpu().numpy()
    preds = preds.data.cpu().numpy()
    # the actual outputs of the model are logits, so we need to pass these values to the sigmoid function
    preds = 1 / (1 + np.exp(-preds))
    test_preds.append(preds)
test_preds = np.hstack(test_preds)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.56it/s]


In [42]:
test_df = pd.read_csv("data/test.csv")
for i, col in enumerate(["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]):
    test_df[col] = test_preds[:, i]

In [43]:
test_df.head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,0.006111,0.004378,0.004602,0.01785,0.018634,0.005039
1,0000247867823ef7,== From RfC == \r\n\r\n The title is fine as i...,0.006111,0.004378,0.004602,0.01785,0.018634,0.005039
2,00013b17ad220c46,""" \r\n\r\n == Sources == \r\n\r\n * Zawe Ashto...",0.006111,0.004378,0.004602,0.01785,0.018634,0.005039


#TODO Bi-LSTM for Linguistic Tasks